### Topics:

    
    Common Table Expressions
        CTE – use common table expressions to make complex queries more readable.
        
    Pivot - PIVOT – convert rows to columns



![MSSQL serverSample DB](SQL-Server-Sample-Database.png)

### CTE

CTE stands for common table expression. A CTE allows you to define a temporary named result set that available temporarily in the execution scope of a statement such as SELECT, INSERT, UPDATE, DELETE, or MERGE.

Syntax:

    WITH expression_name[(column_name [,...])]
    AS
        (CTE_definition)
    SQL_statement;

In this syntax:

    First, specify the expression name (expression_name) to which you can refer later in a query.
    Next, specify a list of comma-separated columns after the expression_name. The number of columns must be the same as the number of columns defined in the CTE_definition.
    Then, use the AS keyword after the expression name or column list if the column list is specified.
    After, define a SELECT statement whose result set populates the common table expression.
    Finally, refer to the common table expression in a query (SQL_statement) such as SELECT, INSERT, UPDATE, DELETE, or MERGE.

In [1]:
import pyodbc
import os
import pandas as pd

#Check if drivers are installed
[x for x in pyodbc.drivers() if x.startswith("Microsoft Access Driver")]

# Define the connection string
conn_str = (
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=localhost;'
    r'DATABASE=BikeStores;'
    r'Trusted_Connection=yes;'
)

# Establish the connection
conn = pyodbc.connect(conn_str)

# Create a cursor
cursor = conn.cursor()

We prefer to use common table expressions rather than to use subqueries because common table expressions are more readable. We also use CTE in the queries that contain analytic functions (or window functions)

This query uses a CTE to return the sales amounts by sales staffs in 2018:

In [2]:
# execute a query
cursor.execute('''
WITH cte_sales_amounts (staff, sales, year) AS (
    SELECT    
        first_name + ' ' + last_name, 
        SUM(quantity * list_price * (1 - discount)),
        YEAR(order_date)
    FROM    
        sales.orders o
    INNER JOIN sales.order_items i ON i.order_id = o.order_id
    INNER JOIN sales.staffs s ON s.staff_id = o.staff_id
    GROUP BY 
        first_name + ' ' + last_name,
        year(order_date)
)

SELECT
    staff, 
    sales
FROM 
    cte_sales_amounts
WHERE
    year = 2018;
''')

# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,staff,sales
0,Genna Serrano,247174.3531
1,Mireya Copeland,230246.9328
2,Kali Vargas,135113.1647
3,Marcelene Boyer,520105.6064
4,Venita Daniel,625358.3947
5,Layla Terrell,56531.3358


In this example:
    
    First, we defined cte_sales_amounts as the name of the common table expression. the CTE returns a result that that consists of three columns staff, year, and sales derived from the definition query.
    Second, we constructed a query that returns the total sales amount by sales staff and year by querying data from the orders, order_items and staffs tables.
    Third, we referred to the CTE in the outer query and select only the rows whose year are 2018.

In [3]:
# execute a query
cursor.execute('''
WITH cte_sales AS (
    SELECT 
        staff_id, 
        COUNT(*) order_count  
    FROM
        sales.orders
    WHERE 
        YEAR(order_date) = 2018
    GROUP BY
        staff_id

)
SELECT
    AVG(order_count) average_orders_by_staff
FROM 
    cte_sales;
''')

# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,average_orders_by_staff
0,48


Rewriting the same query using subquery

In [4]:

# execute a query
cursor.execute('''
SELECT
    AVG(order_count) AS average_orders_by_staff
FROM 
    (
        SELECT 
            staff_id, 
            COUNT(*) AS order_count  
        FROM
            sales.orders
        WHERE 
            YEAR(order_date) = 2018
        GROUP BY
            staff_id
    ) AS subquery;

''')

# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,average_orders_by_staff
0,48


### Pivot

SQL Server PIVOT operator rotates a table-valued expression. It turns the unique values in one column into multiple columns in the output and performs aggregations on any remaining column values.

You follow these steps to make a query a pivot table:

    First, select a base dataset for pivoting.
    Second, create a temporary result by using a derived table or common table expression (CTE)
    Third, apply the PIVOT operator.

In [6]:

# execute a query
cursor.execute('''
SELECT * FROM   
(
    SELECT 
        category_name, 
        product_id
    FROM 
        production.products p
        INNER JOIN production.categories c 
            ON c.category_id = p.category_id
) t 
PIVOT(
    COUNT(product_id) 
    FOR category_name IN (
        [Children Bicycles], 
        [Comfort Bicycles], 
        [Cruisers Bicycles], 
        [Cyclocross Bicycles], 
        [Electric Bikes], 
        [Mountain Bikes], 
        [Road Bikes])
) AS pivot_table;

''')

# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,Children Bicycles,Comfort Bicycles,Cruisers Bicycles,Cyclocross Bicycles,Electric Bikes,Mountain Bikes,Road Bikes
0,59,30,78,10,24,60,60


In [7]:

# execute a query
cursor.execute('''
SELECT * FROM   
(
    SELECT 
        category_name, 
        product_id,
        model_year
    FROM 
        production.products p
        INNER JOIN production.categories c 
            ON c.category_id = p.category_id
) t 
PIVOT(
    COUNT(product_id) 
    FOR category_name IN (
        [Children Bicycles], 
        [Comfort Bicycles], 
        [Cruisers Bicycles], 
        [Cyclocross Bicycles], 
        [Electric Bikes], 
        [Mountain Bikes], 
        [Road Bikes])
) AS pivot_table;

''')

# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,model_year,Children Bicycles,Comfort Bicycles,Cruisers Bicycles,Cyclocross Bicycles,Electric Bikes,Mountain Bikes,Road Bikes
0,2016,3,3,9,2,1,8,0
1,2017,19,10,19,2,2,21,12
2,2018,37,17,50,6,21,31,42
3,2019,0,0,0,0,0,0,6
